In [1]:
import os
import cv2
import csv

import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

import matplotlib.pyplot as plt
%matplotlib inline

In [23]:
num_output = 8
input_shape = (512, 512, 3)

In [24]:
batch_size = 10

In [25]:
IMAGES_FOLDER = '/home/sanjeev309/Projects/posebox/resized_frames'
ANNOTATION_FILE = '/home/sanjeev309/Projects/posebox/annotation_formatted.csv'
OUTPUT = '/home/sanjeev309/Projects/posebox/output'

### Initialise empty numpy arrays

In [26]:
data = np.empty((0,512,512,3), dtype=np.int8)
target = np.empty((0,8), dtype=np.float)

In [27]:
data.shape

(0, 512, 512, 3)

In [28]:
target.shape

(0, 8)

### Read annotation file, fetch image, normalise image and array, compose data and target arrays

In [29]:
with open(ANNOTATION_FILE,'r') as csv_file:
    
    reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    
    for row in reader:
        
        print(row)
        
        if line_count == 0:
            line_count += 1
        else:
            image_path = os.path.join(IMAGES_FOLDER, row[0])
            image = cv2.imread(image_path)/ 255
            image = np.expand_dims(image, axis=0)
            
            points = row[1]
            dimen = (float)(row[2])
            
            p = points.strip('][').split(', ')
            
            
            p = np.array(p, dtype=np.int)
            p = np.divide(p, dimen)
            p = np.expand_dims(p, axis=0)
            
            if image is not None:
                data = np.vstack((data, image))
                target = np.vstack((target, p))
            
            line_count += 1

['file_name', 'coordinates', 'width', 'height']
['image_0.jpg', '[233, 250, 402, 193, 281, 133, 103, 174]', '512', '512']
['image_1.jpg', '[231, 251, 401, 193, 279, 132, 103, 175]', '512', '512']
['image_2.jpg', '[235, 250, 401, 192, 282, 133, 104, 175]', '512', '512']
['image_3.jpg', '[229, 250, 403, 193, 281, 132, 106, 173]', '512', '512']
['image_4.jpg', '[234, 248, 401, 190, 282, 131, 103, 177]', '512', '512']
['image_5.jpg', '[233, 250, 403, 192, 283, 133, 105, 173]', '512', '512']
['image_6.jpg', '[231, 248, 401, 191, 281, 131, 106, 173]', '512', '512']
['image_7.jpg', '[230, 246, 404, 187, 280, 125, 100, 169]', '512', '512']
['image_8.jpg', '[228, 242, 403, 181, 279, 121, 94, 166]', '512', '512']
['image_9.jpg', '[224, 244, 406, 177, 278, 117, 90, 167]', '512', '512']
['image_10.jpg', '[217, 252, 404, 181, 277, 122, 91, 177]', '512', '512']
['image_11.jpg', '[203, 244, 407, 160, 272, 96, 72, 157]', '512', '512']
['image_12.jpg', '[179, 236, 410, 133, 252, 61, 32, 145]', '512', '

### Shuffle data and target synchronously

In [32]:
num_samples = data.shape[0]

In [33]:
arr = np.arange(num_samples)

In [34]:
np.random.shuffle(arr)

In [35]:
arr

array([  4,  50,  28,  98,  27,  82,   1, 101, 100, 106,  33,  43,  94,
        97,  13,  47,  75, 103,  88,  74,  64,  25,   0,  57, 105,  24,
        30,  72,  51,  38,  59,  96,  67,  15,  77,  79,  53,  92,  48,
        69,  10,  63,   3,  61,  29, 104,  18,  35,  32,  22,   8,  21,
        62,  87,  84,  11,  65,  41,  60,  46,  95,  78,  54,  44,  93,
        81,  14,  73,   5,  99,  80,  70,   2, 102,  19,   6,  49,  56,
        20,  71,  36,  17,  39,  58,  12,  83,  34,  76,  45,  86,   7,
       107,  52,   9,  42,  91,  37,  66,  16,  55,  68,  31,  85,  89,
        23,  26,  90,  40])

In [36]:
data = data[arr]
target = target[arr]

In [37]:
print(data.shape)
print(target.shape)

(108, 512, 512, 3)
(108, 8)


In [38]:
np.save(os.path.join(OUTPUT,'data.npy'), data)
np.save(os.path.join(OUTPUT,'target.npy'), target)

### Load data and target

In [39]:
data = np.load(os.path.join(OUTPUT,'data.npy'))
target = np.load(os.path.join(OUTPUT,'target.npy'))

### Train / Test split

In [40]:
TRAIN_RATIO = 0.8

In [41]:
X_train = data[0: int(num_samples * TRAIN_RATIO) - 1]
y_train = target[0: int(num_samples * TRAIN_RATIO) - 1]

X_test = data[int(num_samples * TRAIN_RATIO): num_samples - 1]
y_test = target[int(num_samples * TRAIN_RATIO): num_samples - 1]

In [42]:
print(X_train.shape)

(85, 512, 512, 3)


In [43]:
print(y_train.shape)

(85, 8)


In [44]:
print(X_test.shape)

(21, 512, 512, 3)


In [45]:
print(y_test.shape)

(21, 8)


In [56]:
def build_model():
    model = keras.Sequential([
    
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu",  input_shape=input_shape),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.BatchNormalization(),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.BatchNormalization(),
        layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Activation("relu"),
        layers.SeparableConv2D(128, 3, padding="same"),
        layers.BatchNormalization(),
        layers.Activation("relu"),
        layers.SeparableConv2D(128, 3, padding="same"),
        layers.BatchNormalization(),
        layers.MaxPooling2D(3, strides=2, padding="same"),
        layers.BatchNormalization(),
        layers.Flatten(),
        layers.Dense(64),
        layers.Dense(16),
        layers.Dense(num_output, activation="sigmoid"),
      
    ])
    
    return model

In [57]:
model = build_model()

In [58]:
optimizer = keras.optimizers.RMSprop(0.01)

model.compile(loss=keras.losses.mean_absolute_error,
                optimizer=optimizer,
              metrics=['accuracy'])

In [59]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 510, 510, 32)      896       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 255, 255, 32)      0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 255, 255, 32)      128       
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 253, 253, 64)      18496     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 126, 126, 64)      0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 126, 126, 64)      256       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 124, 124, 128)    

In [60]:
train_history = model.fit(data, target, batch_size=batch_size, validation_split= 0.2, epochs=1000)

Epoch 1/1000
9/9 [==============================] - 163s 18s/step - loss: 0.4578 - accuracy: 0.0698 - val_loss: 0.5267 - val_accuracy: 0.0000e+00
Epoch 2/1000
9/9 [==============================] - 34s 4s/step - loss: 0.4749 - accuracy: 0.0814 - val_loss: 0.5266 - val_accuracy: 0.0000e+00
Epoch 3/1000
9/9 [==============================] - 34s 4s/step - loss: 0.4785 - accuracy: 0.1047 - val_loss: 0.4992 - val_accuracy: 0.1364
Epoch 4/1000
9/9 [==============================] - 34s 4s/step - loss: 0.4875 - accuracy: 0.0930 - val_loss: 0.4992 - val_accuracy: 0.1364
Epoch 5/1000
9/9 [==============================] - 34s 4s/step - loss: 0.4458 - accuracy: 0.1512 - val_loss: 0.4829 - val_accuracy: 0.0909
Epoch 6/1000
9/9 [==============================] - 34s 4s/step - loss: 0.4578 - accuracy: 0.1860 - val_loss: 0.4625 - val_accuracy: 0.1818
Epoch 7/1000
9/9 [==============================] - 34s 4s/step - loss: 0.4595 - accuracy: 0.1163 - val_loss: 0.4611 - val_accuracy: 0.1818
Epoch 8/10

9/9 [==============================] - 34s 4s/step - loss: 0.5252 - accuracy: 0.0349 - val_loss: 0.4959 - val_accuracy: 0.0000e+00
Epoch 59/1000
9/9 [==============================] - 34s 4s/step - loss: 0.5251 - accuracy: 0.0233 - val_loss: 0.4974 - val_accuracy: 0.0000e+00
Epoch 60/1000
9/9 [==============================] - 34s 4s/step - loss: 0.5268 - accuracy: 0.0233 - val_loss: 0.5135 - val_accuracy: 0.0000e+00
Epoch 61/1000
9/9 [==============================] - 34s 4s/step - loss: 0.5239 - accuracy: 0.0465 - val_loss: 0.5157 - val_accuracy: 0.0000e+00
Epoch 62/1000
9/9 [==============================] - 34s 4s/step - loss: 0.5303 - accuracy: 0.0233 - val_loss: 0.5167 - val_accuracy: 0.0000e+00
Epoch 63/1000
9/9 [==============================] - 34s 4s/step - loss: 0.5261 - accuracy: 0.0349 - val_loss: 0.5187 - val_accuracy: 0.0000e+00
Epoch 64/1000
9/9 [==============================] - 34s 4s/step - loss: 0.5275 - accuracy: 0.0349 - val_loss: 0.5249 - val_accuracy: 0.0000e+00

8/9 [=========================>....] - ETA: 5s - loss: 0.5401 - accuracy: 0.0000e+00

KeyboardInterrupt: 